<a href="https://colab.research.google.com/github/SandeepGaddala/Projects/blob/main/Hotel_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

Load the files




In [ ]:
orders = pd.read_csv('/content/orders.csv')
train_locations = pd.read_csv('/content/train_locations.csv')
vendors = pd.read_csv('/content/vendors.csv')


prepare positive sample and drop duplicate data

In [ ]:
positive_samples = orders[['customer_id', 'vendor_id', 'LOCATION_NUMBER']].rename(columns={'LOCATION_NUMBER': 'location_number'})
positive_samples['target'] = 1
positive_samples = positive_samples.drop_duplicates()


In [ ]:
customer_locations = train_locations[['customer_id', 'location_number']].drop_duplicates()
vendor_ids = vendors[['id']].rename(columns={'id': 'vendor_id'})

customer_locations['key'] = 1
vendor_ids['key'] = 1
all_combinations = customer_locations.merge(vendor_ids, on='key').drop('key', axis=1)


Merge Values and assign values

In [ ]:
full_data = all_combinations.merge(
    positive_samples,
    on=['customer_id', 'location_number', 'vendor_id'],
    how='left'
)
full_data['target'] = full_data['target'].fillna(0).astype(int)


Distance between customer and vendor

In [ ]:
import numpy as np

def euclidean_distance(lat1, lon1, lat2, lon2):
    return np.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)


Model Building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# X = features, y = target
X = full_data.drop(columns=['target', 'customer_id', 'vendor_id'])
y = full_data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9864788330000168


In [ ]:
full_data['CID X LOC_NUM X VENDOR'] = (
    full_data['customer_id'] + ' X ' +
    full_data['location_number'].astype(str) + ' X ' +
    full_data['vendor_id'].astype(str)
)

output = full_data[['CID X LOC_NUM X VENDOR', 'target']]
output.to_csv("final_output.csv", index=False)
